# AI演習 第7回
### ディープラーニングによる自然言語処理 (1)



[実行環境]<br>
Colaboratoryの実行環境は更新されるので確認しておきます。<br>
Python: 3.10.12<br>

ランタイムのタイプは、GPUを指定するように注意してください。

In [ ]:
!python --version

Python 3.10.12


### 自然言語処理とは
* 日本語、英語、タイ語など、日常の会話などで使用する言語を自然言語と言います。
自然言語処理とは、自然言語を対象として単語や句の抽出や、係り受けの解析といった処理を経て、コンピュータが機械翻訳、自動要約、文章分類、文脈理解、会話生成などを行えるようにするための処理です。

* 自然言語処理の応用例
 * 文書分類
 * 検索
 * 機械翻訳
 * 文書要約
 * 質問応答
 * 対話
 * 品詞タグ付け
 * 単語分割
 * 語義曖昧性解消
 * 固有表現抽出
 * 構文解析
 * 述語項構造認識



### 自然言語処理とディープラーニング
* トピックモデルによる文書中の単語の出現確率の推定や隠れマルコフモデルによる品詞推定などが行われてきました。

* 2013年にword2vecのような単語の分散表現をニューラルネットワークで学習する方式が考案され、さらにRNNやLSTMが自然言語処理に適用されるとともに、機械翻訳、対話文生成、自動要約、画像説明文の生成などの応用が広がっていきました。

### 言語モデルと文脈
* 単語が文書中に出現する過程を確率過程と見なし、単語がある位置に出現する確率を計算するモデルを言語モデルといいます。また言語モデルにおいて，ある単語の出現確率を計算する際に用いる周囲の単語を文脈といいます。

### 自然言語処理の流れ

1.   データセットの準備
2.   前処理
3.   単語の数値化
4.   データセットの学習 = 応用モデルの構築
5.   応用モデルによる分類や回帰


### 前処理
* 形態素解析、n-gram分割、ストップワードなどの処理をして、文章を解析プログラムが処理しやすい形式に加工することです。

 * 表記の統一
 * 小文字化と大文字化
 * 単語置換



### 形態素解析の実行例

形態素解析ライブラリjanomeを利用した形態素解析の実行例です。最初にjanomeをインストールしています。

In [ ]:
!pip install janome

In [ ]:
from janome.tokenizer import Tokenizer

text = '私は空を見て、あなたは鏡をのぞき込む。'

jt = Tokenizer()
for token in jt.tokenize(text):
  print(token)

In [ ]:
for token in jt.tokenize(text, wakati=True):
  print(token)

#### 1-1 確認例題
次の文章を形態素解析し、区切られた単語ごとに表示してみましょう。

文章: 鏡の中で、あなたは鳥が青空を飛んで行くのを見た。

### 単語の数値化
* 単語間の関連性などを定量的に計算できるように、単語を数値化します。


In [ ]:
text = 'I look at sky and you look in mirror.'

In [ ]:
text = text.lower() # 小文字にする
text = text.replace('.', ' .') # ピリオドを分割
words = text.split(' ') # 空白を区切りにして単語を分割

In [ ]:
print(text)

In [ ]:
print (words)

In [ ]:
def word2id(words):

  word_to_id = {}

  for word in words:
    if word not in word_to_id:
      new_id = len(word_to_id)
      word_to_id[word] = new_id

  return word_to_id

def id2word(word_to_id):
  id_to_word = {}
  for word, id in word_to_id.items():
    id_to_word[id] = word

  return id_to_word

In [ ]:
# 単語の数値化
word_to_id = word2id(words)
print(word_to_id)

# 数値からの逆引き
id_to_word = id2word(word_to_id)
print(id_to_word)

In [ ]:
print(word_to_id['look'])
print(id_to_word[7])

#### 1-2 確認例題
次の文章について、word2id()を使用して、単語をidに変換してみましょう。また、id2word()を使用して、idから対応する単語を抽出してみましょう。

In the mirror, you saw a bird flying across the blue sky.

### One-hotベクトル表現
要素の値が0と1からなり、かつ1か所だけが1のベクトルです。自然言語を処理するニューラルネットワークでは、one-hotベクトルで表現されたものを入力とすることが多いです。

'I look at sky and you look in mirror.'<br>
を上の例のように分解した場合、数値を要素番号とすると各単語のone-hotベクトルを下記のように生成することができます。

i: [1, 0, 0, 0, 0, 0, 0, 0, 0] <br>
look: [0, 1, 0, 0, 0, 0, 0, 0, 0] <br>
at: [0, 0, 1, 0, 0, 0, 0, 0, 0] <br>

In [ ]:
# 単語-数値リストのデータを、one-hot形式に変換
def make_one_hot(corpus):
    N = corpus.shape[0]
    dim =len(word_to_id)

    one_hot = np.zeros((N, dim), dtype=np.int32)
    for idx, word_id in enumerate(corpus):
        one_hot[idx, word_id] = 1

    return one_hot


In [ ]:
import numpy as np
print(words)
corpus = [word_to_id[word] for word in words]
corpus = np.array(corpus)

print(corpus)

make_one_hot(corpus)

#### 1-3 確認例題
次の文章について、one-hotベクトル表現に変換してみましょう。

In the mirror, you saw a bird flying across the blue sky.

### 単語の分散表現（埋め込み表現）
* 単語をベクトルで表現したものです。
* 単語分散表現を得るためのニューラルネットワークを用いた手法として，下記のものがあります。

 * Word2Vec
 * GloVe
 * fastText

* 通常の深層学習を使用して学習させ、単語分散表現を得ることもできます。


### Word2Vec
* 単語の分散表現(埋め込み)を生成する手法

* CBOWモデルとskip gramモデルを使用
* 2013年にTomas Mikolovらが考案
* 分布仮説（単語の意味は周囲の単語によって形成されるという仮説）に基づいて、単語の意味表現を学習する。
* 2層のニューラルネットワークで学習する。隠れ層の重みが単語の分散表現となる。

* 生成された単語の分散表現は、各単語の意味を表したベクトル行列となっており，単語間の距離を計算できる。

 * vector('Paris') - vector('France') + vector('Italy’) = vector(‘Rome’) <br>
 * vector('king') - vector('man') + vector('woman’) = vector('queen')

<br>

#### CBOWモデル

* 複数の単語を文脈として、一つの単語を予測する。文脈単語の順序は問わない。
<br>

<img src='https://drive.google.com/uc?export=view&id=1zGpPVAQ7hBmkjEkeFMNa0p5vHXv192hq' width='60%'>

#### Skip-gramモデル

* 一つの単語を使用してして、複数の単語を文脈として予測する。文脈単語は、入力単語の近さに応じて重みを決める。

<img src='https://drive.google.com/uc?export=view&id=18ipD4BXzaNA9tnggGE8yZTGA0mD70rSB' width='60%'>


#### 1-4 確認例題
Word2VecのCBOWモデルとskip-gramモデルについて、それぞれ単語の分散行列を作成するための学習方法を説明してみましょう。

### 単語の分散表現行列の作成

下記のサイトで提供されるコーパスを用いて、単語の分散表現行列を作成します。

(英語) http://mattmahoney.net/dc/textdata.html <br>
(日本語) https://github.com/Hironsan/ja.text8

In [ ]:
!wget http://mattmahoney.net/dc/text8.zip

In [ ]:
!unzip text8.zip

In [ ]:
import logging
from gensim.models.word2vec import Word2Vec, Text8Corpus

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = Text8Corpus('text8')
model = Word2Vec(sentences, vector_size=100)

model.save('model.bin')

In [ ]:
model = Word2Vec.load('model.bin')

# dogの分散表現ベクトル
print(model.wv['dog'])
print(model.wv['dog'].shape)
# carと意味が似ている単語を抽出
model.wv.most_similar(['car'])

上の説明で用いた下記の式が成り立つかを確認してみます。

vector('Paris') - vector('France') + vector('Italy’) = vector(‘Rome’)


In [ ]:
vector = model.wv['paris'] - model.wv['france'] + model.wv['italy']

model.wv.most_similar(vector)

#### 1-5 確認例題
下記の式が成り立つかを試してみましょう。

vector('king') - vector('man') + vector('woman’) = vector('queen')

#### 1-6 確認例題
日本語コーパス(ja.text8.zip)をダウンロードし、同様の処理をしてみましょう。

In [ ]:
!wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/ja.text8.zip
!unzip ja.text8.zip

In [ ]:
import logging
from gensim.models.word2vec import Word2Vec, Text8Corpus

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = Text8Corpus('ja.text8')
model_ja = Word2Vec(sentences, vector_size=100)

model_ja.save('model.ja.bin')

In [ ]:
model_ja = Word2Vec.load('model.ja.bin')

print(model_ja.wv['犬'])
print(model_ja.wv['犬'].shape)
model_ja.wv.most_similar(['車'])

### ベクトル空間モデル


*   情報検索を行うためのアルゴリズム
*   1970年頃から研究が始まる
 * Salton先生等のSMARTシステムが有名

* ベクトル空間上に検索対象データ，検索語をそれぞれベクトルで表現して配置する．
* 検索対象データ，検索語の類似度をベクトル計算（コサイン，内積，距離等）により求める。
<img src='https://drive.google.com/uc?export=view&id=14u9m4dFPDTmMUa4sMLMErm6CwViEiqpX' width='30%'>


### 文書単語行列
* 文書(d1～dm)中に出現する単語集合(t1～tn)によって，特徴付けられた行列

<img src='https://drive.google.com/uc?export=view&id=1y4V0uPCEPcm0JH_v0-NHWtz0SW4_bGkc' width='30%'>

<br>

* 文書単語行列の例:

<img src='https://drive.google.com/uc?export=view&id=1tEOkD8_BnWLuZxf9iiD1zlVCl8Zn7DAv' width='50%'>

### コサイン計算

<img src='https://drive.google.com/uc?export=view&id=1a226Fn74_d-un3DM-Z_d41yWixYQiYFy' width='60%'>

分散表現ベクトルのコサイン尺度を計算する関数cosine_sim()を作成し、「car」と「truck」の分散表現ベクトルのコサイン尺度を計算してみます。

In [ ]:
# コサインを計算する
def cosine_sim(w1, w2):
  cosine_value = np.dot(model.wv[w1], model.wv[w2]) / (np.linalg.norm(model.wv[w1]) * np.linalg.norm(model.wv[w2]))

  return cosine_value

print (cosine_sim('car','truck'))

# ('truck', 0.7117820978164673)

コサイン尺度は0.7151368と出力され、先ほどの「model.wv.most_similar(['car'])」の結果と同じ値が算出されていることが確認できます。

[('driver', 0.7645907402038574), <br>
 ('cars', 0.7256535291671753), <br>
 ('motorcycle', 0.7231885194778442), <br>
 ('taxi', 0.7163602113723755), <br>
 ('truck', 0.7151368856430054), <br>
 ('vehicle', 0.6943105459213257), <br>
...]

#### 1-7 確認例題
コサイン計算プログラムを利用して、「車」と「電車」の分散表現ベクトルのコサイン尺度を計算してみましょう。

#### 1-8 確認例題
(1)コサイン計算プログラムを利用して、下記式の左辺のベクトルと右辺のベクトルのコサイン尺度を計算しましょう。

(2)計算したコサイン尺度に基づいて、下記式が成立するかを考察しましょう。

vector('Paris') - vector('France') + vector('Italy’) = vector(‘Rome’)

vector('king') - vector('man') + vector('woman’) = vector('queen')

### 文書ベクトルの作成

文書中の単語をベクトル化し、それらの単語ベクトルを足すことで、文章ベクトルを作成します。

文書1: apple, banana, grape, pear, strawberry

文書2: grape, pear, mango, melon, watermelon


In [ ]:
doc_vec1 = model.wv['apple'] + model.wv['banana'] + model.wv['grape'] + model.wv['pear'] + model.wv['strawberry']

print (doc_vec1)

In [ ]:
doc_vec2 = model.wv['grape'] + model.wv['pear'] + model.wv['mango'] + model.wv['melon'] + model.wv['watermelon']

print (doc_vec2)

文書1と文書2のコサイン尺度を算出します。

In [ ]:
 cosine_value = np.dot(doc_vec1, doc_vec2) / (np.linalg.norm(doc_vec1) * np.linalg.norm(doc_vec2))

 print(cosine_value)

#### 1-9 確認例題
コサイン尺度の関数を修正し、2つのベクトルを入力して、コサイン尺度を返す関数cosine_sim_vec(v1, v2)を作成してみましょう。

#### 1-10 確認例題
さらに、次の文書ベクトルを作成します。

(1) 文書1と文書4のコサイン尺度、および、文書4と文書5のコサイン尺度をそれぞれ計算しましょう。

(2) 文書の内容とコサイン尺度の値から、文書1と文書4の類似度(どれくらい似ているかを示す度合い)、および、文書4と文書5の類似度について説明してみましょう。

文書3: peach, melon, banana, strawberry, orange

文書4: mathematics, physics, french, geography, chemistry

文書5: mathematics, chemistry, japanese, geography, english

### 参考文献
* https://code.google.com/archive/p/word2vec/
* Tomas Mikolov, Kai Chen, Greg Corrado, and Jeffrey Dean. Efficient Estimation of Word Representations in Vector Space. In Proceedings of Workshop at ICLR, 2013.
* François Chollet, Deep Learning with Python
* 斎藤 康毅，ゼロから作るDeep Learning 2 ―自然言語処理編
* 中山光樹，機械学習・深層学習による自然言語処理入門 scikit-learnとTensorFlowを使った実践プログラミング